In [2]:
import numpy as np
from PIL import Image
import os
import string
from pickle import dump, load
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Input, Dense, LSTM, Embedding, Dropout, add
from keras.models import Model, load_model
from tqdm import tqdm  # Use tqdm instead of tqdm_notebook for Visual Studio Code

# Ensuring tqdm works smoothly in non-notebook environments
tqdm.pandas()


In [3]:
import os
import string
from pickle import dump, load

# Load the document file into memory
def load_fp(filename):
    with open(filename, 'r') as file:
        text = file.read()
    return text

# Get all images with their captions
def img_capt(filename):
    file = load_fp(filename)
    captions = file.split('\n')
    descriptions = {}
    for caption in captions[:-1]:
        img, caption = caption.split('\t')
        if img[:-2] not in descriptions:
            descriptions[img[:-2]] = [caption]
        else:
            descriptions[img[:-2]].append(caption)
    return descriptions

# Data cleaning function will convert all upper case alphabets to lowercase, removing punctuations and words containing numbers
def txt_clean(captions):
    table = str.maketrans('', '', string.punctuation)
    for img, caps in captions.items():
        for i, img_caption in enumerate(caps):
            img_caption = img_caption.replace("-", " ")
            descp = img_caption.split()
            # Uppercase to lowercase
            descp = [wrd.lower() for wrd in descp]
            # Remove punctuation from each token
            descp = [wrd.translate(table) for wrd in descp]
            # Remove hanging 's and a
            descp = [wrd for wrd in descp if len(wrd) > 1]
            # Remove words containing numbers with them
            descp = [wrd for wrd in descp if wrd.isalpha()]
            # Converting back to string
            img_caption = ' '.join(descp)
            captions[img][i] = img_caption
    return captions

# To build vocab of all unique words
def txt_vocab(descriptions):
    vocab = set()
    for key in descriptions.keys():
        [vocab.update(d.split()) for d in descriptions[key]]
    return vocab

# To save all descriptions in one file
def save_descriptions(descriptions, filename):
    lines = []
    for key, desc_list in descriptions.items():
        for desc in desc_list:
            lines.append(key + '\t' + desc)
    data = "\n".join(lines)
    with open(filename, "w") as file:
        file.write(data)

# Set these paths according to the project folder in your system
dataset_text = "C:\\Users\\USER\\OneDrive\\Documents\\GL4\\PFA\\Flickr8k_text"
dataset_images = "C:\\Users\\USER\\OneDrive\\Documents\\GL4\\PFA\\Flickr8k_Dataset"

# To prepare our text data
filename = os.path.join(dataset_text, "Flickr8k.token.txt")

# Loading the file that contains all data and mapping them into descriptions dictionary 
descriptions = img_capt(filename)
print("Length of descriptions =", len(descriptions))

# Cleaning the descriptions
clean_descriptions = txt_clean(descriptions)

# To build vocabulary
vocabulary = txt_vocab(clean_descriptions)
print("Length of vocabulary =", len(vocabulary))

# Saving all descriptions in one file
save_descriptions(clean_descriptions, "descriptions.txt")


Length of descriptions = 8092
Length of vocabulary = 8422


In [15]:
def extract_features(directory):
    model = Xception(include_top=False, pooling='avg')
    features = {}
    total_files = len(os.listdir(directory))
    print(f"Total files in directory: {total_files}")
    
    for pic in tqdm(os.listdir(directory)):
        file_path = os.path.join(directory, pic)
        if os.path.isfile(file_path):  # Check if the path is a file
            try:
                image = Image.open(file_path)
                image = image.resize((299, 299))
                image = np.array(image)  # Convert the image to a numpy array
                if image.shape[2] == 4:  # Handle images with an alpha channel
                    image = image[..., :3]
                image = np.expand_dims(image, axis=0)  # Add a batch dimension
                image = image / 127.5
                image = image - 1.0  # Normalize image
                feature = model.predict(image)
                features[pic] = feature
            except Exception as e:
                print(f"Error processing {file_path}: {e}")
        else:
            print(f"Skipping non-file item: {file_path}")
    print(f"Total processed images: {len(features)}")
    return features

In [17]:
dataset_images = "C:\\Users\\USER\\OneDrive\\Documents\\GL4\\FINALPFA\\Flickr8k_Dataset\\Flicker8k_Dataset"  # Update this path
features = extract_features(dataset_images)
dump(features, open("features.p", "wb"))

# To directly load the features from the pickle file
features = load(open("features.p", "rb"))

Total files in directory: 8091


  0%|          | 0/8091 [00:00<?, ?it/s]

1/1 [==============================] - 1s 589ms/step


  0%|          | 1/8091 [00:00<1:23:55,  1.61it/s]

1/1 [==============================] - 0s 112ms/step


...

1/1 [==============================] - 1s 738ms/step


100%|█████████▉| 8090/8091 [53:35<00:00,  1.27it/s]

1/1 [==============================] - 1s 735ms/step


100%|██████████| 8091/8091 [53:36<00:00,  2.52it/s]

Total processed images: 8091


In [18]:
def load_photos(filename):
    file = load_fp(filename)
    photos = file.split("\n")[:-1]
    return photos

# Function to load clean descriptions
def load_clean_descriptions(filename, photos):
    file = load_fp(filename)
    descriptions = {}
    for line in file.split("\n"):
        words = line.split()
        if len(words) < 1:
            continue
        image, image_caption = words[0], words[1:]
        if image in photos:
            if image not in descriptions:
                descriptions[image] = []
            desc = ' ' + " ".join(image_caption) + ' '
            descriptions[image].append(desc)
    return descriptions

# Function to load features
def load_features(photos):
    all_features = load(open("features.p", "rb"))  # Load features from a pickle file
    features = {k: all_features[k] for k in photos if k in all_features}  # Select only features of needed photos
    return features


In [19]:
filename = os.path.join(dataset_text, "Flickr_8k.trainImages.txt")
train_imgs = load_photos(filename)

# Load the clean descriptions for the training images
train_descriptions = load_clean_descriptions("descriptions.txt", train_imgs)

# Load the features for the training images
train_features = load_features(train_imgs)

In [20]:
print("Dataset: ", len(train_imgs))
print("Descriptions: train=", len(train_descriptions))
print("Photos: train=", len(train_features))

Dataset:  6000
Descriptions: train= 6000
Photos: train= 6000


In [21]:
def dict_to_list(descriptions):
    all_desc = []
    for key in descriptions.keys():
        all_desc.extend(descriptions[key])
    return all_desc

# Create tokenizer class and fit on the descriptions
def create_tokenizer(descriptions):
    desc_list = dict_to_list(descriptions)
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(desc_list)
    return tokenizer

# Calculate the maximum length of descriptions
def max_length(descriptions):
    desc_list = dict_to_list(descriptions)
    return max(len(d.split()) for d in desc_list)

In [22]:
tokenizer = create_tokenizer(train_descriptions)
dump(tokenizer, open('tokenizer.p', 'wb'))

vocab_size = len(tokenizer.word_index) + 1
print("Vocab size:", vocab_size)  # Print the size of vocabulary

max_length = max_length(train_descriptions)
print("Max length of description:", max_length)  # Print max length of description

Vocab size: 7318
Max length of description: 33


In [24]:
def create_sequences(tokenizer, max_length, desc_list, feature):
    x1, x2, y = [], [], []
    for desc in desc_list:
        seq = tokenizer.texts_to_sequences([desc])[0]
        for i in range(1, len(seq)):
            in_seq, out_seq = seq[:i], seq[i]
            in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
            out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
            x1.append(feature)
            x2.append(in_seq)
            y.append(out_seq)
    return np.array(x1), np.array(x2), np.array(y)

# Data generator
def data_generator(descriptions, features, tokenizer, max_length):
    while True:
        for key, desc_list in descriptions.items():
            feature = features[key][0]
            in_img, in_seq, out_word = create_sequences(tokenizer, max_length, desc_list, feature)
            yield [[in_img, in_seq], out_word]

# Example usage
# Assuming train_descriptions, train_features, tokenizer, and max_length are already defined
[a, b], c = next(data_generator(train_descriptions, train_features, tokenizer, max_length))
print(a.shape, b.shape, c.shape)
# Expected output shape: (batch_size, 2048), (batch_size, max_length), (batch_size, vocab_size)

(37, 2048) (37, 33) (37, 7318)


In [25]:
def define_model(vocab_size, max_length):
    # Features from the CNN model compressed from 2048 to 256 nodes
    inputs1 = Input(shape=(2048,))
    fe1 = Dropout(0.5)(inputs1)
    fe2 = Dense(256, activation='relu')(fe1)
    
    # LSTM sequence model
    inputs2 = Input(shape=(max_length,))
    se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
    se2 = Dropout(0.5)(se1)
    se3 = LSTM(256)(se2)
    
    # Merging both models
    decoder1 = add([fe2, se3])
    decoder2 = Dense(256, activation='relu')(decoder1)
    outputs = Dense(vocab_size, activation='softmax')(decoder2)
    
    # Define the model
    model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    # Summarize the model
    print(model.summary())
    plot_model(model, to_file='model.png', show_shapes=True)
    
    return model

In [27]:
from keras.utils import plot_model


def define_model(vocab_size, max_length):
    # Features from the CNN model compressed from 2048 to 256 nodes
    inputs1 = Input(shape=(2048,))
    fe1 = Dropout(0.5)(inputs1)
    fe2 = Dense(256, activation='relu')(fe1)
    
    # LSTM sequence model
    inputs2 = Input(shape=(max_length,))
    se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
    se2 = Dropout(0.5)(se1)
    se3 = LSTM(256)(se2)
    
    # Merging both models
    decoder1 = add([fe2, se3])
    decoder2 = Dense(256, activation='relu')(decoder1)
    outputs = Dense(vocab_size, activation='softmax')(decoder2)
    
    # Define the model
    model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    # Summarize the model
    print(model.summary())
    plot_model(model, to_file='model.png', show_shapes=True)
    
    return model

In [31]:
def generate_desc(model, tokenizer, photo, max_length):
    in_text = 'startseq'
    for i in range(max_length):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=max_length)
        yhat = model.predict([photo, sequence], verbose=0)
        yhat = np.argmax(yhat)
        word = tokenizer.index_word.get(yhat)
        if word is None:
            break
        in_text += ' ' + word
        if word == 'endseq':
            break
    return in_text

In [35]:
from nltk.translate.bleu_score import corpus_bleu
def evaluate_model(model, descriptions, photos, tokenizer, max_length):
    actual, predicted = [], []
    for key, desc_list in descriptions.items():
        yhat = generate_desc(model, tokenizer, photos[key], max_length)
        references = [d.split() for d in desc_list]
        actual.append(references)
        predicted.append(yhat.split())
    print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
    print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
    print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
    print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))

In [37]:
from keras.callbacks import ModelCheckpoint


print('Dataset: ', len(train_imgs))
print('Descriptions: train=', len(train_descriptions))
print('Photos: train=', len(train_features))
print('Vocabulary Size:', vocab_size)
print('Description Length: ', max_length)
model = define_model(vocab_size, max_length)

# Set the number of epochs and steps per epoch
epochs = 10
steps = len(train_descriptions)

# Create a directory to save models if it doesn't exist
if not os.path.exists("models"):
    os.mkdir("models")

# Set up the model checkpoint to save the best model
checkpoint = ModelCheckpoint("models/best_model.h5", monitor='loss', verbose=1, save_best_only=True, mode='min')
# Train the model using a data generator
for i in range(epochs):
    generator = data_generator(train_descriptions, train_features, tokenizer, max_length)
    model.fit(generator, epochs=1, steps_per_epoch=steps, verbose=1, callbacks=[checkpoint])
    model.save(f"models/model_{i}.h5")
    print(f"Epoch {i+1}/{epochs} completed")
    evaluate_model(model, train_descriptions, train_features, tokenizer, max_length)

Dataset:  6000
Descriptions: train= 6000
Photos: train= 6000
Vocabulary Size: 7318
Description Length:  33
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_8 (InputLayer)        [(None, 33)]                 0         []                            
                                                                                                  
 input_7 (InputLayer)        [(None, 2048)]               0         []                            
                                                                                                  
 embedding (Embedding)       (None, 33, 256)              1873408   ['input_8[0][0]']             
                                                                                                  
 dropout (Dropout)           (None, 2048)                 0         ['input_7[0][0]'] 

NameError: name 'ModelCheckpoint' is not defined